## Adım 0 — SE’nin yaptığı şey (tek cümle)

* SE, her kanal için 0–1 arası bir ağırlık üretir ve feature map’i kanala göre çarpar.
* Girdi/Çıktı: (B,C,H,W) -> (B,C,H,W) (shape değişmez)

-----

## Adım 1 — En saf SE (sadece tanım)

Klasik SE: GlobalAvgPool -> FC -> ReLU -> FC -> Sigmoid -> Scale

In [2]:
import torch
import torch.nn as nn

class SeBlock(nn.Module):
    def __init__(self, channels:int , reduction : int = 16):
        super().__init__()

        hidden = max(1,channels//reduction)

        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Conv2d(channels,hidden,kernel_size=1,bias=True)
        self.act = nn.ReLU(inplace=True)
        self.fc2 = nn.Conv2d(hidden,channels,kernel_size=1,bias=True)
        self.gate = nn.Sigmoid()

    def forward(self,x:torch.Tensor) -> torch.Tensor:
        w = self.pool(x)
        w = self.fc1(w)
        w = self.act(w)
        w = self.fc2(w)
        w = self.gate(w)        # (B,C,1,1) in [0,1]
        return x * w

In [3]:
x = torch.randn(2, 64, 56, 56)
se = SeBlock(64, reduction=16)
y = se(x)
print(x.shape, y.shape)


torch.Size([2, 64, 56, 56]) torch.Size([2, 64, 56, 56])


---

## Adım 2 — SE’yi bir Conv bloğa tak (gerçek kullanım)

SE tek başına değil, genelde bir Conv bloğun sonuna konur:

In [4]:
class ConvBNAct(nn.Module):
    def __init__(self,cin,cout,k=3,s=1,p=1):
        super().__init__()

        self.net = nn.Sequential(
            nn.Conv2d(cin,cout,k,stride=s,padding=p,bias=False),
            nn.BatchNorm2d(cout),
            nn.ReLU(inplace=True))
    
    def forward(self,x) : return self.net(x)

class ConvSEBlock(nn.Module):
    def __init__(self, cin,cout,reductions = 16):
        super().__init__()
        self.conv = ConvBNAct(cin,cout,3,1,1)
        self.se = SeBlock(cout,reduction=reductions)

    def forward(self,x):
        x = self.conv(x)
        x = self.se(x)
        return x

----

## Adım 3 — “Daha iyi” SE: aktivasyonu SiLU yap (stabil + pratik)

SE içindeki ReLU yerine SiLU/Swish çoğu modern modelde daha iyi davranır.

In [5]:
class SEv2(nn.Module):
    def __init__(self, channels: int, reduction: int = 16):
        super().__init__()
        hidden = max(1, channels // reduction)

        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc1  = nn.Conv2d(channels, hidden, 1, bias=True)
        self.act  = nn.SiLU(inplace=True)   # değişti
        self.fc2  = nn.Conv2d(hidden, channels, 1, bias=True)
        self.gate = nn.Sigmoid()

    def forward(self, x):
        w = self.pool(x)
        w = self.fc1(w)
        w = self.act(w)
        w = self.fc2(w)
        w = self.gate(w)
        return x * w


----

## Adım 4 — “Daha iyi” SE: AvgPool + MaxPool (birleşik squeeze)

Bazı görevlerde max bilgisi de yararlı olur. CBAM gibi düşün: squeeze’i güçlendir.

In [6]:
class SE_AvgMax(nn.Module):
    def __init__(self, channels: int, reduction: int = 16):
        super().__init__()
        hidden = max(1, channels // reduction)

        self.avg = nn.AdaptiveAvgPool2d(1)
        self.max = nn.AdaptiveMaxPool2d(1)

        self.fc1  = nn.Conv2d(channels, hidden, 1, bias=True)
        self.act  = nn.ReLU(inplace=True)
        self.fc2  = nn.Conv2d(hidden, channels, 1, bias=True)
        self.gate = nn.Sigmoid()

    def forward(self, x):
        s = self.avg(x) + self.max(x)  # squeeze güçlendi
        w = self.fc1(s)
        w = self.act(w)
        w = self.fc2(w)
        w = self.gate(w)
        return x * w


---

## Adım 5 — “Daha pratik” SE: reduction’u otomatik seç (küçük kanalda bozulmasın)

Küçük C değerlerinde C//16 çok küçük kalabiliyor. Bu sürüm hidden’ı güvenli seçer.

In [7]:
class SE_Safe(nn.Module):
    def __init__(self, channels: int, reduction: int = 16, min_hidden: int = 4):
        super().__init__()
        hidden = max(min_hidden, channels // reduction)

        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Conv2d(channels, hidden, 1, bias=True),
            nn.ReLU(inplace=True),
            nn.Conv2d(hidden, channels, 1, bias=True),
            nn.Sigmoid()
        )

    def forward(self, x):
        return x * self.fc(self.pool(x))


----
----
----

## Şimdi son 4 adımı inceleyelim.Daha net ve daha işlevsel hale getirelim.

----
-----
----

# 1) Kapasite ve stabilite: Safe hidden + SiLU

#### Ne değişti?

* **hidden = max(min_hidden, C // reduction) (küçük kanalda kapasite ölmesin)**

* ReLU → SiLU (daha stabil/pratik)

Kod farkı (kritik satırlar):
```python 
hidden = max(min_hidden, channels // reduction)   # yeni
self.act = nn.SiLU(inplace=True)                  # ReLU yerine
```


In [8]:
import torch
import torch.nn as nn

class SE_Stable(nn.Module):
    def __init__(self, channels: int, reduction: int = 16, min_hidden: int = 4):
        super().__init__()
        hidden = max(min_hidden, channels // reduction)   # CHANGED

        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc1  = nn.Conv2d(channels, hidden, 1, bias=True)
        self.act  = nn.SiLU(inplace=True)                # CHANGED
        self.fc2  = nn.Conv2d(hidden, channels, 1, bias=True)
        self.gate = nn.Sigmoid()

    def forward(self, x):
        w = self.pool(x)
        w = self.fc1(w)
        w = self.act(w)
        w = self.fc2(w)
        w = self.gate(w)
        return x * w


# 2) Squeeze’i güçlendir: AvgPool + MaxPool

### Ne değişti?

* Sadece avgpool(x) yerine avg + max kullanıyoruz.

* Bu, kanal özetini daha güçlü yapar.

Kod farkı (kritik satırlar):

```python
self.avg = nn.AdaptiveAvgPool2d(1)   # yeni
self.mx  = nn.AdaptiveMaxPool2d(1)   # yeni
s = self.avg(x) + self.mx(x)         # CHANGED
```


In [9]:
class SE_AvgMax(nn.Module):
    def __init__(self, channels: int, reduction: int = 16, min_hidden: int = 4):
        super().__init__()
        hidden = max(min_hidden, channels // reduction)

        self.avg = nn.AdaptiveAvgPool2d(1)   # ADDED
        self.mx  = nn.AdaptiveMaxPool2d(1)   # ADDED

        self.fc1  = nn.Conv2d(channels, hidden, 1, bias=True)
        self.act  = nn.SiLU(inplace=True)
        self.fc2  = nn.Conv2d(hidden, channels, 1, bias=True)
        self.gate = nn.Sigmoid()

    def forward(self, x):
        s = self.avg(x) + self.mx(x)         # CHANGED (squeeze güçlendi)
        w = self.fc1(s)
        w = self.act(w)
        w = self.fc2(w)
        w = self.gate(w)
        return x * w


# 3) Gate’i pratikleştir: Sigmoid ↔ Hardsigmoid seçilebilir

### Ne değişti?

Gate fonksiyonunu parametre yaptık:

* sigmoid: klasik

* hardsigmoid: genelde daha hızlı (özellikle CPU/edge)

Kod farkı (kritik satırlar):
```python
if gate == "sigmoid": self.gate = nn.Sigmoid()
elif gate == "hardsigmoid": self.gate = nn.Hardsigmoid()


In [10]:
class SE_Gated(nn.Module):
    def __init__(self, channels: int, reduction: int = 16, min_hidden: int = 4, gate: str = "sigmoid"):
        super().__init__()
        hidden = max(min_hidden, channels // reduction)

        self.avg = nn.AdaptiveAvgPool2d(1)
        self.mx  = nn.AdaptiveMaxPool2d(1)

        self.fc1 = nn.Conv2d(channels, hidden, 1, bias=True)
        self.act = nn.SiLU(inplace=True)
        self.fc2 = nn.Conv2d(hidden, channels, 1, bias=True)

        g = gate.lower()
        if g == "sigmoid":
            self.gate = nn.Sigmoid()
        elif g == "hardsigmoid":
            self.gate = nn.Hardsigmoid()
        else:
            raise ValueError("gate 'sigmoid' veya 'hardsigmoid' olmalı.")

    def forward(self, x):
        s = self.avg(x) + self.mx(x)
        w = self.fc1(s)
        w = self.act(w)
        w = self.fc2(w)
        w = self.gate(w)
        return x * w


# 4) Entegrasyon ve kontrol: Squeeze türünü aç/kapat (ablation-friendly)

### Ne değişti?

* use_avg, use_max ile squeeze türlerini kontrol ediyoruz.

Bu sayede:

* classification: sadece avg çoğu zaman yeter

* detection/seg: avg+max bazen daha iyi

Kod farkı (kritik satırlar):
```python
assert use_avg or use_max
s = 0.0
if use_avg: s += avg(x)
if use_max: s += max(x)


# Hepsini birleştiren “SEPlusPlus” (Final)

In [ ]:
class SEPlusPlus(nn.Module):
    def __init__(
        self,
        channels: int,
        reduction: int = 16,
        min_hidden: int = 4,
        gate: str = "sigmoid",      # "sigmoid" | "hardsigmoid"
        use_avg: bool = True,
        use_max: bool = True,
    ):
        super().__init__()
        assert use_avg or use_max, "En az bir squeeze türü açık olmalı."

        # (1) Safe hidden
        hidden = max(min_hidden, channels // reduction)

        # (4) Ablation-friendly squeeze
        self.use_avg = use_avg
        self.use_max = use_max
        self.avg = nn.AdaptiveAvgPool2d(1)
        self.mx  = nn.AdaptiveMaxPool2d(1)

        # (1) SiLU
        self.fc1 = nn.Conv2d(channels, hidden, 1, bias=True)
        self.act = nn.SiLU(inplace=True)
        self.fc2 = nn.Conv2d(hidden, channels, 1, bias=True)

        # (3) Gate choice
        g = gate.lower()
        if g == "sigmoid":
            self.gate = nn.Sigmoid()
        elif g == "hardsigmoid":
            self.gate = nn.Hardsigmoid()
        else:
            raise ValueError("gate 'sigmoid' veya 'hardsigmoid' olmalı.")

    def forward(self, x):
        # (2) Avg + Max squeeze (opsiyonel)
        s = 0.0
        if self.use_avg:
            s = s + self.avg(x)
        if self.use_max:
            s = s + self.mx(x)

        w = self.fc1(s)
        w = self.act(w)
        w = self.fc2(w)
        w = self.gate(w)
        return x * w

----
----
## Gelin son dokunuşlarımızı yapalım.
---
---



# SE++ (Geliştirme Planı)

Bu hücrede, mevcut SEPlusPlus modülünü daha stabil ve ablation-friendly hale getirmek için 3 iyileştirme ekleyeceğiz:

## (A) Avg/Max Squeeze Birleştirme: Öğrenilebilir Ağırlıklar
- `avg(x)` ve `max(x)` çıktılarını sabit toplamak yerine,
- `w_avg` ve `w_max` ile **öğrenilebilir** birleştireceğiz.
- Ağırlıkları **softmax** ile normalize edeceğiz: `softmax([w_avg, w_max])`
- Böylece gate'in saturate olma ihtimali azalır ve model hangi squeeze'in daha faydalı olduğunu öğrenir.

## (B) Temperature (T) ile Gate Saturasyonunu Kontrol
- Sigmoid girişini yumuşatmak için: `gate_input / T`
- `T > 1` daha yumuşak, `T < 1` daha keskin gate üretir.
- Varsayılan olarak `T=1.0` kullanıp ablation ile deneyebiliriz.
- İster sabit ister öğrenilebilir yapılabilir (burada opsiyon sunacağız).

## (C) Residual Gating (Feature Öldürmeyi Azaltır)
- Klasik SE: `y = x * gate`
- Residual SE: `y = x * (1 + alpha * gate)`
- `alpha` öğrenilebilir olursa model residual etkisini kendisi ayarlar.
- Bu, özellikle detection/segmentation tarafında stabiliteye yardımcı olabilir.


# Ablation Checklist

Denenecek varyantlar:
1) Squeeze:
   - sadece avg
   - sadece max
   - avg+max (öğrenilebilir fusion)

2) Gate:
   - sigmoid
   - hardsigmoid

3) Temperature:
   - sabit T=1.0
   - sabit T=2.0
   - learnable T (pozitif olacak şekilde)

4) Residual:
   - kapalı: y = x * gate
   - açık:  y = x * (1 + alpha * gate)
     - alpha sabit (örn. 1.0)
     - alpha learnable


In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class SEPlusPlusV2(nn.Module):
    def __init__(
        self,
        channels: int,
        reduction: int = 16,
        min_hidden: int = 4,
        gate: str = "sigmoid",          # "sigmoid" | "hardsigmoid"
        use_avg: bool = True,
        use_max: bool = True,
        fusion: str = "softmax",        # "sum" | "softmax"
        temperature: float = 1.0,
        learnable_temperature: bool = False,
        residual: bool = False,
        alpha_init: float = 1.0,
        learnable_alpha: bool = False,
        bias: bool = True,
        eps: float = 1e-6,
    ):
        super().__init__()
        assert use_avg or use_max, "En az bir squeeze türü açık olmalı."
        assert fusion in ("sum", "softmax"), "fusion 'sum' veya 'softmax' olmalı."
        assert temperature > 0, "temperature pozitif olmalı."

        self.channels = channels
        self.use_avg = use_avg
        self.use_max = use_max
        self.fusion = fusion
        self.residual = residual
        self.eps = eps

        # Safe hidden
        hidden = max(min_hidden, channels // reduction)

        # Squeeze ops
        self.avg = nn.AdaptiveAvgPool2d(1)
        self.mx = nn.AdaptiveMaxPool2d(1)

        # Learnable fusion weights (only meaningful if both enabled)
        # Start from equal importance.
        if self.use_avg and self.use_max:
            self.fusion_logits = nn.Parameter(torch.zeros(2))  # [0,0] -> softmax = [0.5,0.5]
        else:
            self.fusion_logits = None

        # Excitation MLP (1x1 conv)
        self.fc1 = nn.Conv2d(channels, hidden, kernel_size=1, bias=bias)
        self.act = nn.SiLU(inplace=True)
        self.fc2 = nn.Conv2d(hidden, channels, kernel_size=1, bias=bias)

        # Gate choice
        g = gate.lower()
        if g == "sigmoid":
            self.gate_fn = torch.sigmoid
        elif g == "hardsigmoid":
            self.gate_fn = F.hardsigmoid
        else:
            raise ValueError("gate 'sigmoid' veya 'hardsigmoid' olmalı.")

        # Temperature: fixed or learnable (positive)
        self.learnable_temperature = learnable_temperature
        if learnable_temperature:
            # parametrize T as softplus(t_raw) + eps to keep strictly positive
            # init so that softplus(t_raw) ~= temperature
            t_raw = torch.tensor(float(temperature))
            # inverse softplus approx: log(exp(x)-1)
            t_inv = torch.log(torch.exp(t_raw) - 1.0 + eps)
            self.t_raw = nn.Parameter(t_inv)
        else:
            self.register_buffer("T", torch.tensor(float(temperature)))

        # Residual alpha
        self.learnable_alpha = learnable_alpha
        if residual:
            if learnable_alpha:
                self.alpha = nn.Parameter(torch.tensor(float(alpha_init)))
            else:
                self.register_buffer("alpha", torch.tensor(float(alpha_init)))

    def _get_temperature(self) -> torch.Tensor:
        if self.learnable_temperature:
            return F.softplus(self.t_raw) + self.eps
        return self.T

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # Squeeze
        s_list = []
        if self.use_avg:
            s_list.append(self.avg(x))
        if self.use_max:
            s_list.append(self.mx(x))

        if len(s_list) == 1:
            s = s_list[0]
        else:
            # two squeezes: avg + max
            if self.fusion == "sum":
                s = s_list[0] + s_list[1]
            else:
                # softmax fusion: a*avg + b*max
                w = torch.softmax(self.fusion_logits, dim=0)  # shape (2,)
                s = w[0] * s_list[0] + w[1] * s_list[1]

        # Excite
        w = self.fc1(s)
        w = self.act(w)
        w = self.fc2(w)

        # Temperature + gate
        T = self._get_temperature()
        w = w / T
        w = self.gate_fn(w)

        # Apply
        if self.residual:
            return x * (1.0 + self.alpha * w)
        return x * w
